# ORIE 5256 Numerai Tournament

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Initialize NumerAPI - the official Python API client for Numerai
from numerapi import NumerAPI
napi = NumerAPI()

# list the datasets and available versions
all_datasets = napi.list_datasets()
dataset_versions = list(set(d.split('/')[0] for d in all_datasets))
print("Available versions:\n", dataset_versions)

# Set data version to one of the latest datasets
DATA_VERSION = "v5.0"

# Print all files available for download for our version
current_version_files = [f for f in all_datasets if f.startswith(DATA_VERSION)]
print("Available", DATA_VERSION, "files:\n", current_version_files)

Available versions:
 ['v5.0']
Available v5.0 files:
 ['v5.0/features.json', 'v5.0/live.parquet', 'v5.0/live_benchmark_models.parquet', 'v5.0/live_example_preds.csv', 'v5.0/live_example_preds.parquet', 'v5.0/meta_model.parquet', 'v5.0/train.parquet', 'v5.0/train_benchmark_models.parquet', 'v5.0/validation.parquet', 'v5.0/validation_benchmark_models.parquet', 'v5.0/validation_example_preds.csv', 'v5.0/validation_example_preds.parquet']


## 1. Feature Engineering

We will use the `medium` feature set offer by Numerai. This feature set contains a total of 705 features. In this section, we will perform some feature engineering methods to ensure the stationarity of the data, and to reduce the dimensionality to avoid curse of dimensionality.

In [ ]:
import json

napi = NumerAPI()  # initialize API client
DATA_VERSION = 'v5.0'

# Load metadata
napi.download_dataset(f'{DATA_VERSION}/features.json')
feature_metadata = json.load(open(f'{DATA_VERSION}/features.json'))
feature_sets = feature_metadata['feature_sets']
medium_features = feature_sets['medium']

# Load training data
napi.download_dataset(f'{DATA_VERSION}/train.parquet')
train_set = pd.read_parquet(f'{DATA_VERSION}/train.parquet', columns=['era', 'target'] + medium_features)

# Downsample to every 4th era
train_set = train_set[train_set['era'].isin(train_set['era'].unique()[::4])]

2024-11-19 14:43:41,010 INFO numerapi.utils: target file already exists
2024-11-19 14:43:41,012 INFO numerapi.utils: download complete
2024-11-19 14:43:41,682 INFO numerapi.utils: target file already exists
2024-11-19 14:43:41,683 INFO numerapi.utils: download complete


In [ ]:
train_set.head()

,era,target,feature_able_deprived_nona,feature_ablest_inflexional_egeria,feature_absorbable_hyperalgesic_mode,feature_accoutered_revolute_vexillology,feature_acetose_crackerjack_needlecraft,feature_acheulian_conserving_output,feature_acronychal_bilobate_stevenage,feature_acrylic_gallic_wine,...,feature_working_jain_acromegaly,feature_wrapround_chrestomathic_timarau,feature_xanthic_transpadane_saleswoman,feature_xanthochroid_petrified_gutenberg,feature_zincy_cirrhotic_josh,feature_zippy_trine_diffraction,feature_zonal_snuffly_chemism,feature_zygotic_middlebrow_caribbean,feature_zymolytic_intertidal_privet,feature_zymotic_windswept_cooky
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,0001,0.25,1,2,3,2,3,2,2,2,...,2,0,3,2,4,3,2,1,0,0
n003bba8a98662e4,0001,0.25,3,2,4,1,0,2,3,2,...,2,0,0,2,0,0,2,0,0,0
n003bee128c2fcfc,0001,0.75,1,2,0,2,4,2,0,2,...,2,3,3,2,2,3,2,2,2,4
n0048ac83aff7194,0001,0.25,1,2,3,4,0,2,3,2,...,2,0,2,1,1,4,2,0,2,1
n0055a2401ba6480,0001,0.25,3,2,3,4,1,2,4,2,...,2,1,3,3,2,4,2,4,1,3


### 1.1 Stationarity

In [ ]:
pass

### 1.2 Low Mutual Information

In this part, we filter out those features that are highly correlated with each other.

In [ ]:
# Calculate pairwise correlations between features. Drop one from each highly correlated pari (threshold = .8)

correlation_matrix = train_set[medium_features].corr().abs()
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
train_set.drop(columns=to_drop, inplace=True)

In [ ]:
train_set.to_parquet(f'train_set_low_corr.parquet')

In [ ]:
# train_set = pd.read_parquet('train_set_low_corr.parquet')

In [ ]:
# Store ne wfeatures
low_corr_features = list(train_set.columns[2:])

### 1.3 Dimension Reduction

We will use Principal Component Analysis (PCA) to reduce the dimensionality of the data. The first 100 principal components will be kept.

In [ ]:
# Apply PCA to the features and store the first 100 components

from sklearn.decomposition import PCA
pca = PCA(n_components=100)
# fit PCA to the features
pca_X = pca.fit_transform(train_set[low_corr_features])
# store the PCA features in the training set
pca_features = [f'pca_{i}' for i in range(100)]  # name of the pca features
df_pca_features = pd.DataFrame(pca_X, index=train_set.index, columns=pca_features)
train_set = pd.concat([train_set, df_pca_features], axis=1)

In [ ]:
train_set.drop(columns=low_corr_features, inplace=True)
train_set.head()

,era,target,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,...,pca_90,pca_91,pca_92,pca_93,pca_94,pca_95,pca_96,pca_97,pca_98,pca_99
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,0001,0.25,-5.638687,0.978715,-0.874781,2.758017,-1.400958,2.404050,4.439399,-1.285807,...,0.702926,-0.824793,-2.119851,-0.597447,-1.077121,0.225363,1.236070,-0.577384,-2.413472,0.564858
n003bba8a98662e4,0001,0.25,-4.207500,-5.707142,-1.375227,2.484462,-3.858972,2.208746,-0.112917,-1.173767,...,-1.165892,-0.084253,-1.588472,1.297324,1.505793,-1.381338,1.061128,1.000957,2.961764,-0.862889
n003bee128c2fcfc,0001,0.75,0.378066,7.492630,-1.348457,2.809567,1.541221,-1.288045,0.037517,0.956088,...,0.072736,0.068531,-0.722200,0.077919,-3.083180,0.544806,0.532066,-0.938183,0.926706,0.811019
n0048ac83aff7194,0001,0.25,-0.027852,-7.763445,-2.374904,-2.657793,5.039552,0.549712,2.198034,2.407041,...,1.768557,0.583036,1.427570,-0.063548,-0.648705,1.912037,-0.164595,0.371036,0.134518,0.551601
n0055a2401ba6480,0001,0.25,-2.958113,-3.687835,-2.035751,0.456909,3.160161,2.932720,-5.245039,3.349961,...,0.646266,-0.301437,-0.713286,-0.901061,-0.281857,0.957327,-0.442464,0.427004,0.877403,2.157457


## Feature Selection

We will use the Mean Decrease Accuracy (MDA) analysis to select the most important features. For this multi-class classification problem, our baseline classifier is Random Forest. We will use Purged K-Fold Cross Validation with AUC-ROC as scoring metric. Features with positive mean score improvement will be kept.

In [ ]:
train_set['era'] = train_set['era'].astype(int)

In [ ]:
# Construct inputs

t1 = pd.Series((train_set['era'] + 4).values, index=train_set['era'])
X = train_set[pca_features].copy()
X.index = t1.index
y = train_set['target'].copy()
y.index = t1.index
y = y.astype(str)

In [ ]:
# Compute sample weights
from sklearn.utils.class_weight import compute_sample_weight
sample_weight = compute_sample_weight(class_weight='balanced', y=train_set['target'])
sample_weight = pd.Series(sample_weight, index=train_set.index)

In [ ]:
from sklearn.model_selection._split import _BaseKFold

class PurgedKFold(_BaseKFold):
    """Extend KFold class to work with labels that span intervals.

    The train is purged of observations overlapping test-label intervals.
    Test set is assumed contiguous (shuffle=False), w/o training samples in between.
    """

    def __init__(self, n_splits=3, t1=None, pctEmbargo=0.0):
        """Initialize PurgedKFold object.

        Args:
            n_splits (int): Number of splits. Default is 3.
            t1 (pd.Series):
                t1.index: time when the observation started
                t1.value: time when the observation ended
            pctEmbargo (float): Percentage of embargo on test set. Embargo step = pctEmbargo * T. Default is 0.
        """
        if not isinstance(t1, pd.Series):
            raise ValueError('Label Through Dates must be a pd.Series')
        super(PurgedKFold, self).__init__(
            n_splits, shufﬂe=False, random_state=None
        )

        self.t1 = t1
        self.pctEmbargo = pctEmbargo

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.

        Args:
            X (pd.DataFrame): Features.
            y (pd.Series): Labels.
            groups: Ignored.
        """
        if (X.index == self.t1.index).sum() != len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')

        indices = np.arange(X.shape[0])

        mbrg = int(X.shape[0] * self.pctEmbargo)
        test_starts = [
            (i[0], i[-1] + 1)
            for i in np.array_split(np.arange(X.shape[0]), self.n_splits)
        ]
        for test_start, test_end in test_starts:
            t0 = self.t1.index[test_start]   # start of test set
            test_indices = indices[test_start: test_end]

            max_t1 = self.t1.iloc[test_indices].max()
            maxT1Idx = self.t1.index.searchsorted(self.t1.iloc[test_indices].max())
            train_indices = list(t1[t1 <= t0].reset_index(drop=True).index)
            if maxT1Idx < X.shape[0]:   # right train (with embargo)
                train_indices = np.concatenate(
                    (train_indices, indices[maxT1Idx + mbrg :])
                )
            yield train_indices, test_indices

In [ ]:
def featImpMDA(
    clf, X, y, cv, sample_weight, t1, pctEmbargo, scoring='auc-roc'
):
    """feat importance based on OOS score reduction"""
    if scoring not in ['auc-roc']:
        raise Exception('wrong scoring method.')
    from sklearn.metrics import roc_auc_score

    cvGen = PurgedKFold(
        n_splits=cv, t1=t1, pctEmbargo=pctEmbargo
    )   # purged cv
    scr0 = pd.Series()
    scr1 = pd.DataFrame(columns=X.columns)

    for i, (train, test) in enumerate(cvGen.split(X=X)):
        X0, y0, w0 = X.iloc[train, :], y.iloc[train], sample_weight.iloc[train]
        X1, y1, w1 = X.iloc[test, :], y.iloc[test], sample_weight.iloc[test]
        fit = clf.fit(X=X0, y=y0, sample_weight=w0.values)
        if scoring == 'auc-roc':
            prob = fit.predict_proba(X1)
            scr0.loc[i] = roc_auc_score(
                y1, prob, sample_weight=w1.values, labels=clf.classes_, multi_class='ovr', average='macro'
            )
        else:
            raise Exception('Only auc-roc scoring is supported')
        for j in X.columns:
            X1_ = X1.copy(deep=True)
            np.random.shuffle(X1_[j].values)   # permutation of a single column
            if scoring == 'auc-roc':
                prob = fit.predict_proba(X1_)
                scr1.loc[i, j] = roc_auc_score(
                    y1, prob, sample_weight=w1.values, labels=clf.classes_, multi_class='ovr', average='macro'
                )
            else:
                raise Exception('Only auc-roc scoring is supported')
    imp = (-scr1).add(scr0, axis=0)
    if scoring == 'auc-roc':
        imp = imp / (1.0 - scr1)
    else:
        raise Exception('Only auc-roc scoring is supported')
    imp = pd.concat(
        {'mean': imp.mean(), 'std': imp.std() * imp.shape[0] ** -0.5}, axis=1
    )
    return imp, scr0.mean()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_features=int(1))
imp, scr0mean = featImpMDA(
    clf,
    X=X,
    y=y,
    cv=5,
    sample_weight=sample_weight,
    t1=t1,
    pctEmbargo=0.01,
)

In [ ]:
# Find features with import mean > 0
imp_pca_features = list(imp[imp['mean'] > 0].index)

83

In [ ]:
train_set_selected = train_set[['era', 'target'] + imp_pca_features]